In [1]:
! pip install pandas
! pip install pydicom

import glob, pylab, pandas as pd
import pydicom, numpy as np
from skimage.transform import resize



    100% |████████████████████████████████| 8.9MB 173kB/s ta 0:00:011    22% |███████▏                        | 2.0MB 2.9MB/s eta 0:00:03    28% |█████████▏                      | 2.5MB 1.6MB/s eta 0:00:05    44% |██████████████                  | 3.9MB 3.0MB/s eta 0:00:02    52% |████████████████▉               | 4.7MB 3.2MB/s eta 0:00:02    72% |███████████████████████▎        | 6.5MB 2.9MB/s eta 0:00:01
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 7.3MB 241kB/s eta 0:00:01
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# Show files in data
!ls data

# display label format
df = pd.read_csv('data/stage_1_train_labels.csv')


# 
No = 4
patientId = df['patientId'][No]
dcm_file = 'data/stage_1_train_images/%s.dcm' % patientId
dcm_data = pydicom.read_file(dcm_file)
#print(df.iloc[No])


#print(np.unique(df.Target))
#print(df['patientId'])

img_size = [224,224]
new_df = pd.DataFrame({'Target':[],'Image':[]})
unq, idx = np.unique(df['patientId'], return_index = True)
print(len(idx))
for i in idx:  # len(np.unique(df['patientId']))
    new_df.Target[i] = df.Target[i]
    patientId = df['patientId'][i]
    dcm_file = 'data/stage_1_train_images/%s.dcm' % patientId
    dcm_data = pydicom.read_file(dcm_file)
    new_df.Image[i] = resize(dcm_data.pixel_array, output_shape=img_size, mode='reflect', anti_aliasing=True) 
    if not i % 100:
        print(i)
new_df.to_csv('data/training_df.csv')    



#print(dcm_data)

#

im = new_df.Image[30]
im = resize(im, output_shape=img_size, mode='reflect', anti_aliasing=True) 
pylab.imshow(im, cmap=pylab.cm.gist_gray)
pylab.axis('off')

GCP Credits Request Link - RSNA.txt  stage_1_test_images
note.txt			     stage_1_train_images
stage_1_detailed_class_info.csv      stage_1_train_labels.csv
stage_1_sample_submission.csv
25684
0
100
200
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2100
2200
2400
2700
2800
3000
3100
3300
3400
3500
3700
3900
4100
4300
4700
4800
4900
5000
5100
5200
5300
5400
5600
5700
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400


In [ ]:
dcm_data. 